# Proyecto 2
## Problema Minimum Coin Change

**Importar librerias**

In [9]:
import sys

## Implementación del algoritmo con Dynamic Programming
El resultado del algoritmo es una matriz donde el eje vertical (eje y) representa las denominaciones de las monedas, ordenadas de menor a mayor. El valor en una casilla con coordenadas (x, y) representa el número mínimo de monedas necesarias para formar el valor x utilizando todas las monedas con denominaciones menores o iguales a y. Por lo tanto, el resultado final del algoritmo es el número ubicado en la esquina inferior derecha de la matriz. Este número indica el mínimo número de monedas necesarias para formar el valor deseado con las monedas disponibles.

In [10]:
def init_table(coins: [int], amount: int):
    table = [[0]*(amount+1) for _ in range(len(coins))]

    for i in range(len(table[0])):
        table[0][i] = i

    return table

# the amounts MUST include a 1 cent coin
# the coins list MUST be sorted
def coin_change(coins: [int], amount: int):    
    table = init_table(coins, amount)

    for row in range(1, len(coins)):
        for col in range(amount+1):
            if col < coins[row]:
                table[row][col] = table[row-1][col]
            else:
                table[row][col] = min(table[row-1][col], 1 + table[row][col-coins[row]])

    return table


def print_result(coins: [int], amount: int):
    table = coin_change(coins, amount)

    print("x", end="  ")
    for i in range(amount+1):
        print(i, end="  ")
    print()

    for i in range(len(table)):
        print(coins[i], end=" ")
        print(table[i])

coins = [1,3,4,5]
amount = 7
table = print_result(coins, amount)


x  0  1  2  3  4  5  6  7  
1 [0, 1, 2, 3, 4, 5, 6, 7]
3 [0, 1, 2, 1, 2, 3, 2, 3]
4 [0, 1, 2, 1, 1, 2, 2, 2]
5 [0, 1, 2, 1, 1, 1, 2, 2]


### Implementacion con Programacion dinamica V2

In [11]:
def minCoinChangeDP(coin, m, K):
    # If the total amount K is 0, return 0
    if K == 0:
        return 0

    # Initialize Change array with large positive values (as infinity)
    Change = [sys.maxsize] * (K + 1)
    Change[0] = 0  # Base case: 0 coins needed for the total amount of 0

    # Fill the Change array with the minimum number of coins for each amount up to K
    for i in range(1, K + 1):
        for j in range(m):
            if coin[j] <= i:
                currCount = Change[i - coin[j]]
                if currCount != sys.maxsize and currCount + 1 < Change[i]:
                    Change[i] = currCount + 1

    # If Change[K] is still sys.maxsize, it means the amount K cannot be formed by any combination of coins
    if Change[K] == sys.maxsize:
        return -1
    else:
        return Change[K]

In [12]:
# Testing the function
# List of coin denominations
coins = [1, 3, 4, 5]

# Parameters
m = len(coins)
K = 7

# Function call
print(minCoinChangeDP(coins, m, K))

2


## Implementacion con Divide y Conquista

In [13]:
def minCoinChangeDAC(coin, m, K):
    # Base case: When the total amount K is 0, 0 coins are needed
    if K == 0:
        return 0

    # Initialize minCount to a very large positive value
    minCount = sys.maxsize

    # Try every coin that has a value less than or equal to K
    for i in range(m):
        if coin[i] <= K:
            # Recursively find the number of coins needed for the remaining amount
            currCount = minCoinChangeDAC(coin, m, K - coin[i])

            # If the number of coins needed for the remaining amount is not infinite,
            # and is less than the current minCount, update minCount
            if currCount != sys.maxsize and currCount + 1 < minCount:
                minCount = currCount + 1

    # If minCount has not been updated, return -1 to indicate failure
    if minCount == sys.maxsize:
        return -1
    else:
        return minCount

In [14]:
# Testing the function
# List of coin denominations
coins = [1, 3, 4, 5]

# Parameters
m = len(coins)
K = 7

# Function call
print(minCoinChangeDAC(coins, m, K))

2


**Referencias:**
- https://www.enjoyalgorithms.com/blog/minimum-coin-change
- https://www.boardinfinity.com/blog/coin-change-problem-with-dp-and-recursion-approach/